In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import cv2


In [ ]:
#!pip install opencv-contrib-python 

In [ ]:
# list mp4 filenames to import
mp4FileName = []
#mp4FileName.append('207R_090419')
#mp4FileName.append('207RR_090419')
#mp4FileName.append('207LL_090419')
#mp4FileName.append('366L_090419')
mp4FileName.append('366B_090419')
#mp4FileName.append('367B_090419')

# list 100x100 pixel ROI anchors (frames = 31095, y = 720, x = 1280, RGBcolor = 3)
ROIanchors = []
#ROIanchors.append([275,580])
#ROIanchors.append([320,650])
#ROIanchors.append([300,650])
#ROIanchors.append([330,650])
ROIanchors.append([330,540])
#ROIanchors.append([330,540])


In [ ]:

#sys.getsizeof(x)   # get the size of a variable

# Note, takes like 15 seconds per 1000 frames

#vidIDX = 3
ROIsize = 100

for vidIDX in range(0,len(mp4FileName)):

    cap = cv2.VideoCapture(mp4FileName[vidIDX] + '.mp4')
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    #buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    buf = np.empty((frameCount, ROIsize, ROIsize), np.dtype('uint8'))

    # example pixel dimensions (frames = 31095, y = 720, x = 1280, RGBcolor = 3)  (video is ~1870 seconds long)

    fc = 0
    ret = True

    while (fc < frameCount  and ret):  # fc < frameCount, fc < 1000

        # read in a given frame    (shape of tempFrame = (720, 1280, 3))
        ret, tempFrame = cap.read()

        # compress 3 colors dimensions into 1 grayscale dimension
        tempFrame = np.mean(tempFrame,axis=2)

        # use y and x anchor to crop frame
        tinyFrame = tempFrame[ROIanchors[vidIDX][0]:ROIanchors[vidIDX][0]+ROIsize,ROIanchors[vidIDX][1]:ROIanchors[vidIDX][1]+ROIsize]

        # discretize pixel bins (32 bins, not 256 (uint8))
        tinyFrame = tinyFrame*3
        tinyFrame[tinyFrame > 255] = 255
        tinyFrame = np.floor(tinyFrame/8)

        buf[fc] = tinyFrame

        #ret, buf[fc] = cap.read()
        fc += 1

        if fc%10000 == 0:

            print(fc)

    cap.release()

    #### save buf ??
    #np.save(mp4FileName[vidIDX]+'.npy', buf)    # .npy extension is added if not given



In [ ]:
#bufavg = np.mean(buf[0:1000,:,:],axis=0)

# Explore images from the data set
plt.figure()
plt.figure(figsize=(14,14))

for panel in range(9):
    
    #bufavg = np.std(buf[(panel*1000+1):((panel+1)*1000),:,:],axis=0)
    bufavg = np.mean(buf[(panel*1000+1):((panel+1)*1000),:,:],axis=0)
    plt.subplot(3, 3, panel+1)
    plt.imshow(bufavg)
    
    #plt.colorbar()
    plt.grid(False)
    
plt.show()
# Check size of buf (in GB)
print('Size of buf (GB): {}'.format(sys.getsizeof(buf) / 10**9))